In [1]:
import os
import pandas as pd
# import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# Load the train , test and validation data and labels
print(os.listdir("../../data/raw/Food"))
labels_df = pd.read_csv("../../data/raw/Food/labels/labels.csv")
# Define the data transformations
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

['val', 'train', 'labels', 'test']


In [3]:
# Custom dataset class
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, dataframe, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        # Load image files
        self.image_files = sorted(
            [f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f))]
        )
        # Initialize a dictionary to map frame identifiers to labels
        self.labels_map = {}
        # Populate the labels_map
        for _, row in dataframe.iterrows():
            self.labels_map[row["Frame_Number"]] = row["Label"]
        # Filter out image files without a corresponding label
        self.image_files = [
            img
            for img in self.image_files
            if os.path.splitext(img)[0] in self.labels_map
        ]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        full_img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(full_img_path).convert("RGB")

        frame_identifier = os.path.splitext(img_name)[0]
        label = self.labels_map.get(frame_identifier)

        # Handle the unlikely case where a label is not found
        if label is None:
            print(f"Warning: Label not found for image: {img_name}. Skipping...")
            return None  # This should be handled by your dataloader or skipped

        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
# Update your DataLoader to skip None types (which we use for missing labels)
from torch.utils.data.dataloader import default_collate


def collate_fn(batch):
    batch = [b for b in batch if b is not None]
    return default_collate(batch)


train_data_path = "../../data/raw/Food/train"
test_data_path = "../../data/raw/Food/test"
val_data_path = "../../data/raw/Food/val"

train_dataset = CustomImageDataset(train_data_path, labels_df, transform)
test_dataset = CustomImageDataset(
    test_data_path, labels_df, transform
)  # Adjust these according to actual splits
val_dataset = CustomImageDataset(
    val_data_path, labels_df, transform
)  # Adjust these according to actual splits

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# !pip install efficientnet_pytorch
# !pip install optuna

In [6]:
from efficientnet_pytorch import EfficientNet

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# Assuming you have torchvision installed
# from torchvision.models import EfficientNet
import optuna

# Check for GPU availability and use it if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Define the training function
def train_model(trial, train_loader, val_loader, num_epochs=5):
    # Sample hyperparameters from the trial
    learning_rate = trial.suggest_float("learning_rate", 1e-7, 0.1)
    optimizer_name = trial.suggest_categorical(
        "optimizer", ["Adam", "SGD", "RMSprop", "Adamax", "Adagrad", "Adadelta"]
    )
    num_epochs = trial.suggest_int("num_epochs", 500, 1500)

    # Initialize and move the model to the specified device
    model = EfficientNet.from_name("efficientnet-b0", num_classes=2).to(device)

    # Define loss function
    criterion = nn.CrossEntropyLoss()

    # Define optimizer based on the sampled name
    optimizer = {
        "Adam": optim.Adam(model.parameters(), lr=learning_rate),
        "SGD": optim.SGD(model.parameters(), lr=learning_rate),
        "RMSprop": optim.RMSprop(model.parameters(), lr=learning_rate),
        "Adamax": optim.Adamax(model.parameters(), lr=learning_rate),
        "Adagrad": optim.Adagrad(model.parameters(), lr=learning_rate),
        "Adadelta": optim.Adadelta(model.parameters(), lr=learning_rate)
    }[optimizer_name]

    # Lists to store training and validation losses
    train_losses = []
    valid_losses = []

    # Train the model
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            # Move data to the device
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Calculate average training loss for the epoch
        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        # Evaluate the model on the validation set
        model.eval()  # Set the model to evaluation mode
        valid_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                valid_loss += loss.item()

        # Calculate average validation loss for the epoch
        valid_loss /= len(val_loader)
        valid_losses.append(valid_loss)

        # Report intermediate results to Optuna
        trial.report(valid_loss, epoch)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        # Print training statistics
        print(
            f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}"
        )

    return valid_losses[-1]

# Define the objective function for Optuna
def objective(trial):
    # You can pass additional arguments to `train_model` if needed
    return train_model(trial, train_loader, val_loader)

# Create a study object and optimize hyperparameters
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=17)

# Get the best hyperparameters and train the final model with them
best_params = study.best_params
# final_val_loss = train_model(best_params, train_loader, val_loader, num_epochs=1500)

print("Best hyperparameters:", best_params)
# print("Final validation loss with best hyperparameters:", final_val_loss)


/home/olarinoyem/miniconda3/envs/deep_tf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-04-16 15:39:58,449] A new study created in memory with name: no-name-650f6c80-39d5-4495-95ab-48c1af59bb9d


Using device: cuda
Epoch [1/882], Train Loss: 0.6320, Valid Loss: 0.8281
Epoch [2/882], Train Loss: 0.3778, Valid Loss: 0.8713
Epoch [3/882], Train Loss: 0.2597, Valid Loss: 1.0981
Epoch [4/882], Train Loss: 0.1452, Valid Loss: 1.3289
Epoch [5/882], Train Loss: 0.0957, Valid Loss: 2.0660
Epoch [6/882], Train Loss: 0.0595, Valid Loss: 2.6005
Epoch [7/882], Train Loss: 0.0925, Valid Loss: 1.7431
Epoch [8/882], Train Loss: 0.0754, Valid Loss: 2.5662
Epoch [9/882], Train Loss: 0.0370, Valid Loss: 1.9615
Epoch [10/882], Train Loss: 0.0432, Valid Loss: 1.8273
Epoch [11/882], Train Loss: 0.0359, Valid Loss: 1.0574
Epoch [12/882], Train Loss: 0.0476, Valid Loss: 0.5115
Epoch [13/882], Train Loss: 0.0442, Valid Loss: 0.2824
Epoch [14/882], Train Loss: 0.0219, Valid Loss: 0.1710
Epoch [15/882], Train Loss: 0.0328, Valid Loss: 0.2698
Epoch [16/882], Train Loss: 0.0152, Valid Loss: 0.2669
Epoch [17/882], Train Loss: 0.0135, Valid Loss: 0.1844
Epoch [18/882], Train Loss: 0.0332, Valid Loss: 0.1678


[W 2024-04-16 16:50:05,839] Trial 0 failed with parameters: {'learning_rate': 0.004581446189538255, 'optimizer': 'Adamax', 'num_epochs': 882} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/olarinoyem/miniconda3/envs/deep_tf/lib/python3.11/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_860080/3534238321.py", line 96, in objective
    return train_model(trial, train_loader, val_loader)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_860080/3534238321.py", line 46, in train_model
    for i, (images, labels) in enumerate(train_loader):
  File "/home/olarinoyem/miniconda3/envs/deep_tf/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 633, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/home/olarinoyem/miniconda3/envs/deep_tf/lib/python3.11/site-packag

KeyboardInterrupt: 

In [ ]:
import pickle

# Assuming 'study' is your Optuna study object
with open("study.pkl", "wb") as f:
    pickle.dump(study, f)